<a href="https://colab.research.google.com/github/GuilhermeVeriss/practice/blob/main/python_learn/firstNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np

In [162]:
# Small Sample

def even_odd(n):
  if n % 2 == 0:
    return 1
  return 0

inputsample = np.array([np.random.randint(0, 100) for _ in range(500)])
outputsample = np.array(list(map(even_odd, inputsample)))

train = np.array([inputsample, outputsample])

In [209]:
class NeuralNet:
  def __init__(self, laysizes):
    self.layers = len(laysizes)
    self.laysizes = laysizes
    self.biases = np.array([np.random.randn(y) for y in laysizes[1:]])
    self.weights = np.array([np.random.randn(y, x) for x, y in zip(laysizes[:-1], laysizes[1:])])
  
  def sigmoid(self, z):
    return 1 / (1 + np.exp(-z))
  
  def diff_sigmoid(self, z):
    return self.sigmoid(z)*(1-self.sigmoid(z))

  def feedforward(self, a):
    for b, w in zip(self.biases, self.weights):
      a = self.sigmoid((w @ a) + b)
    return a

  def SGD(self, train_data, learn_rate, samplesize, epochs):
    mini_batches_input = []
    mini_batches_expect = []

    for _ in range(epochs):

      # splits the train_data into mini_batches
      for k in range(0, len(train_data[0])-samplesize, samplesize):
        mini_batches_input.append(train_data[0, k:k+samplesize])
        mini_batches_expect.append(train_data[1, k:k+samplesize])
      
      for batch in range(len(mini_batches_input)):
        self.update_network(mini_batches_input[batch], 
                            mini_batches_expect[batch], learn_rate)
  

  def update_network(self, inp, outp, learn_rate):
    # calculates the mean gradient of C and updates the NeuralNetwork's weights and bias
    n = len(inp)

    nabla_w = np.array([np.zeros(w.shape) for w in self.weights])
    nabla_b = np.array([np.zeros(len(b)) for b in self.biases])

    for x, y in zip(inp, outp): 
      wi, bi = self.backprop(x, y)
      nabla_w += wi
      nabla_b += bi
    
    nabla_w /= n
    nabla_b /= n

    self.weights += -learn_rate * nabla_w
    self.biases += -learn_rate * nabla_b



  def backprop(self, x, y):
      nabla_b = np.array([np.zeros(len(b)) for b in self.biases])
      nabla_w = np.array([np.zeros(w.shape) for w in self.weights])

      a = x
      activations = [x]
      z_matrix = []

      for b, w in zip(self.biases, self.weights):
        print(f"w= {w}")
        z = w @ a
        print(f"z = {z}")
        z += b
        print(f"z+b = {z}")
        z_matrix.append(z)
        a = self.sigmoid(z)
        activations.append(a)
      
      # 1° Backpropagation equation
      delta = (activations[-1] - y) * self.diff_sigmoid(z_matrix[-1])

      # 3° and 4° Backprop equations (nabla w and nabla b)
      nabla_b[-1] = delta
      nabla_w[-1] = delta @ activations[-2].T

  
      for i in range(2, self.layers-1):
        # 2° Backpropagation equation
        delta = self.weights[-i+1].T @ delta *\
                 self.diff_sigmoid(z_matrix[-i])
        nabla_b[-i] = delta
        nabla_w[-i] = delta @ activations[-i-1].T
      
      return nabla_w, nabla_b
        


In [210]:
n1 = NeuralNet([1, 3, 1])
n1.SGD(train, 0.01, 100, 1)

w= [[ 0.50847216]
 [ 0.18819227]
 [-0.31823516]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


ValueError: ignored

In [185]:
k = 33
print(n1.feedforward([k]))

[array([-0.07971133,  1.08024619,  1.5944893 ])
 array([[-0.6429444 , -0.6429799 , -0.64298752]])]
[[0.01971162 0.01971093 0.01971079]]


In [51]:
n1.weights

array([array([[-1.543798  ],
              [ 0.29024501]]), array([[-0.20615767, -1.18282391]])],
      dtype=object)

In [52]:
n1.biases

array([array([[-0.90927836],
              [ 1.08163457]]), array([[-0.62092278]])], dtype=object)